In [1]:
!pip install pysentimiento


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 11.0 MB/s 
     |████████████████████████████████| 312 kB 47.1 MB/s 
     |████████████████████████████████| 175 kB 54.6 MB/s 
     |████████████████████████████████| 3.3 MB 56.7 MB/s 
     |████████████████████████████████| 880 kB 74.0 MB/s 
     |████████████████████████████████| 163 kB 69.3 MB/s 
     |████████████████████████████████| 115 kB 73.4 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 127 kB 75.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=e6c0ed48fdb86235cc4e487dcc79c49dd88fa41fd7a07ba00b32b0a64ff96417
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=f15ca55acb3fd3f8538bb9c325b15979282

In [2]:
import pandas as pd
from pysentimiento import create_analyzer

In [3]:
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import Data

In [4]:
df = pd.read_csv('/content/drive/MyDrive/nlp-tweets-classification/tweets_traducidos.csv')
df.head(2)

,Unnamed: 0,id_tweet,date,author,text,app,id_user,followers,following,stauses,...,retweeted_id,user_retweeted,quoted_id,user_quoted,first_HT,lang,link,is_rt,label,texto_traducido
0,0,1530089428147916802,2022-05-27 07:32:02,@ConcelloCoruna,RT @coruna_deportes: 📣 487 preinscricións en a...,Twitter for Android,3332841207,44595,53,28696,...,1530081368943644673,coruna_deportes,None,None,CoruñaNoMar,es,https://twitter.com/ConcelloCoruna/status/1530...,True,NaN,NaN
1,1,1529842783053238273,2022-05-26 15:11:57,@ConcelloCoruna,🗓️ O encontro terá lugar nos xardíns de Méndez...,Twitter Web App,3332841207,44595,53,28696,...,None,None,None,None,None,es,https://twitter.com/ConcelloCoruna/status/1529...,False,NaN,NaN


With the next line join the tweets that are in spanish with the tweets that are in other language and we traduced to spanish

In [5]:
all_text = [df.text[i] if pd.isnull(df.texto_traducido[i]) else df.texto_traducido[i] for i in range(len(df.texto_traducido))]

In [6]:
df['all_text'] = all_text

In [7]:
df_1 = df[['id_tweet', 'all_text']]


df_1 = df_1.dropna()

# Inicialize Analyzers

In [8]:
sentiment = create_analyzer(task="sentiment", lang="es")
#emotion = create_analyzer(task="emotion", lang="en")
#hate_speech = create_analyzer(task="hate_speech", lang="es")

#analyzers = [sentiment, emotion, hate_speech]

Downloading:   0%|          | 0.00/925 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/415M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/334 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/838k [00:00<?, ?B/s]

First we will make prediction of sentiment analysis for all the town halls tweets

In [9]:
preds = [sentiment.predict(df_1.all_text[i]) for i in tqdm(range(len(df_1.all_text)))]

  0%|          | 0/470367 [00:00<?, ?it/s]

In [10]:
df_1['sentiment'] = [preds[i].output for i in tqdm(range(len(preds)))]

  0%|          | 0/470367 [00:00<?, ?it/s]

In [11]:
probas_df = pd.DataFrame()

for i in tqdm(range(len(preds))):

  probas_df = pd.concat([probas_df,pd.DataFrame(preds[i].probas, index=[0])], axis=0)

probas_df.reset_index(drop=True, inplace=True)
probas_df

  0%|          | 0/470367 [00:00<?, ?it/s]

,NEG,NEU,POS
0,0.062692,0.842473,0.094835
1,0.137418,0.670005,0.192578
2,0.020112,0.177067,0.802820
3,0.051358,0.850937,0.097705
4,0.081412,0.471540,0.447048
...,...,...,...
470362,0.041357,0.887010,0.071633
470363,0.131497,0.791350,0.077153
470364,0.007182,0.399394,0.593424
470365,0.219945,0.748685,0.031370


In [12]:
df_1 = pd.concat([df_1,probas_df], axis=1)

In [14]:
df_1

,id_tweet,all_text,sentiment,NEG,NEU,POS
0,1530089428147916802,RT @coruna_deportes: 📣 487 preinscricións en a...,NEU,0.062692,0.842473,0.094835
1,1529842783053238273,🗓️ O encontro terá lugar nos xardíns de Méndez...,NEU,0.137418,0.670005,0.192578
2,1529842773280571392,"🗣️ @jmlage, en la presentación del VI Encuent...",POS,0.020112,0.177067,0.802820
3,1529813385558339587,🚀 #Ecosystems2030 terá lugar os vindeiros días...,NEU,0.051358,0.850937,0.097705
4,1529813382253117441,"📢 A alcaldesa, @inesreygarcia, recibiu hoxe no...",NEU,0.081412,0.471540,0.447048
...,...,...,...,...,...,...
470362,1405801286184931328,#Zaragoza apuesta por el control biológico de ...,NEU,0.041357,0.887010,0.071633
470363,1405786697128976386,¡Buenos días #Zaragoza! #TiempoZGZ ☁ Claros ...,NEU,0.131497,0.791350,0.077153
470364,1405586036311347207,Llega la Ruta de la Tapa de #Goya Una iniciati...,POS,0.007182,0.399394,0.593424
470365,1405563390446030851,"¿Buscas animal de compañía? No compres, ¡adopt...",NEU,0.219945,0.748685,0.031370


In [15]:
df_1.to_csv('/content/drive/MyDrive/nlp-tweets-classification/town_halls_sentiment_analysis.csv', index=False)

In [17]:
df_1.sentiment.value_counts(normalize=True)

NEU    0.811073
POS    0.169565
NEG    0.019361
Name: sentiment, dtype: float64

With this loop make all the analysis at same time

In [ ]:
df_analyzers = pd.DataFrame()

for n,a in tqdm(enumerate(analyzers)):
  if n == 0:
    name = 'sentiment'
  elif n == 1:
    name = 'emotion'
  else:
    name = 'hate_speech'
  
  print (f'Starting {name} analysis\n')

  preds = [a.predict(df.text[i]) for i in tqdm(range(len(df.text)))]
  
  df_analyzers[name] = [preds[i].output for i in tqdm(range(len(preds)))]
  probas_df = pd.DataFrame()

  for i in tqdm(range(len(preds))):

    probas_df = pd.concat([probas_df,pd.DataFrame(preds[i].probas, index=[0])], axis=0)

  probas_df.reset_index(drop=True, inplace=True)

  df_analyzers = pd.concat([df_analyzers, probas_df], axis=1)

  df_analyzers.reset_index(drop=True, inplace=True)

  print(f'End {name} analysis')
  print('\n-----------\n')

@misc{perez2021pysentimiento,
      title={pysentimiento: A Python Toolkit for Sentiment Analysis and SocialNLP tasks},
      author={Juan Manuel PÃ©rez and Juan Carlos Giudici and Franco Luque},
      year={2021},
      eprint={2106.09462},
      archivePrefix={arXiv},
      primaryClass={cs.CL}
}